# 第二届·2022量子精选论文复现挑战赛 第 07 题
## 论文题目：
## QAOA-in-QAOA: solving large-scale MaxCut problems on small quantum machines
## 论文链接：https://arxiv.53yu.com/abs/2205.11762
### 复现人：Waikikilick

### 论文简介：
为组合优化问题设计快速算法对物流、金融和化学等领域都有很大的价值。 量子近似组合优化算法（Quantum Approximate Optimization Algorithm，QAOA）在这类问题的求解上相比经典算法有很好的加速效果。但是，受限于当前时期量子系统的大小，QAOA 在计算大型问题时还是很困难。本文提出一种“分而治之”的方法，可以用小型量子系统求解大型问题。以图最大割为研究对象，该方法将大图分割成一个个可以被量子系统处理的小图。在对小图用 QAOA 求解后，再以各小图作为顶点，组成一个新的小图，就又形成一个新的可用 QAOA 求解的最大割问题。所以，该方法被称为 QAOA in QAOA，或者 QAOA$^2$。如果采用多层嵌套，可以用小型量子计算机，求解任意大图的最大割问题。



关于 QAOA 的基础知识和在 MindQuantum 中的实现，可参考 MindQuantum 的系列教程之 [量子近似优化算法
](https://www.mindspore.cn/mindquantum/docs/zh-CN/master/quantum_approximate_optimization_algorithm.html)。

<img src='./src/sample.jpg' width='90%'>

这是一个由 13 个顶点构成的无向图。假设只可以使用 4 量子比特的量子计算机，则采用 QAOA in QAOA 算法求解该图最大割的流程可分为五步：

### 第一步：分割
将大图 $G$ 分割为多个小图或称子图。每个子图的顶点数不超过量子计算机的规模，即 4。 如图可分割为 4 个子图 $G_1$，$G_2$，$G_3$ 和 $G_4$。
### 第二步：子图求解
对每个子图分别进行 QAOA 求解最大割。
### 第三步：以子图为顶点，组成新图
以每个子图为顶点，构成一个新图。该新图的边由各子图之间的边组成。
### 第四步：对新图求解
用 QAOA 算法对新图进行求解。
### 第五步：求最终解
最终解即为各个子图的最大割数加上新图的最大割数。

如果图很大，可能需要重复上面步骤，如图 (c)。

## 案例讲解：

接下来，我们通过一个案例来展示算法流程。作为范例，我们考虑和示意图相同的图结构进行分析，该图的边为:

In [52]:
edges = [(0,1), (0,12), (1,2), (2,3), (1,3), (0,3), (3,5), (2,4), (4,5), (4,6), (4,7), (5,11), (5,7), (5,6), (6,7), (7,8), (8,9), (9,10), (10,12), (10,11), (11,12)]

我们可以将图中的点随机分组，每组最大数不超过量子比特数 4。

In [5]:
import random
nodes = list(range(13)) 
qubits_num = 4
graphs_nodes = []
while True:
    sub_graph_nodes = random.sample(nodes, qubits_num)
    graphs_nodes.append(sub_graph_nodes)
    for i in sub_graph_nodes:
        nodes.remove(i)
    if len(nodes) <= qubits_num:
        graphs_nodes.append(nodes)
        break
        
print(graphs_nodes)

[[8, 2, 0, 5], [3, 4, 1, 12], [9, 6, 11, 10], [7]]


但作为示范案例，我们还是采用示意图中的切分方案。根据该方案，图共拆分成四个子图：

In [53]:
graphs_nodes = [[0,1,2,3], [4,5,6,7], [8,9], [10,11,12]]

根据子图顶点，挑出各个子图内所包含的边:

In [54]:
graphs_edges = []

for nodes in graphs_nodes:
    sub_graph_edges = []
    for edge in edges:
        if edge[0] in nodes and edge[1] in nodes:
            sub_graph_edges.append(edge)
    graphs_edges.append(sub_graph_edges)
    
print(graphs_edges)

[[(0, 1), (1, 2), (2, 3), (1, 3), (0, 3)], [(4, 5), (4, 6), (4, 7), (5, 7), (5, 6), (6, 7)], [(8, 9)], [(10, 12), (10, 11), (11, 12)]]


将每个子图的顶点都重新映射为一个新图的顶点，这样就可以在量子计算中只使用很少的量子比特：

In [102]:
rename_graphs_nodes = [] # 所有子图中的新索引
rename_nodes_list_0 = [] # 新索引：顶点序号
rename_nodes_list_1 = [] # 顶点序号：新索引

for sub_graph_nodes in graphs_nodes:
    rename_graphs_nodes_0 = {} 
    rename_graphs_nodes_1 = {} 
    rename_node = []
    for index, node in enumerate(sub_graph_nodes):
        rename_node.append(index)
        rename_graphs_nodes_0[index] = node
        rename_graphs_nodes_1[node] = index
    rename_nodes_list_0.append(rename_graphs_nodes_0)
    rename_nodes_list_1.append(rename_graphs_nodes_1)
    rename_graphs_nodes.append(rename_node)

print(graphs_nodes)
print(rename_graphs_nodes) 
print(rename_nodes_list_0) # 正向映射  # 正反定义随意，但强调了两者之间关系
print(rename_nodes_list_1) # 反向映射

[[0, 1, 2, 3], [4, 5, 6, 7], [8, 9], [10, 11, 12]]
[[0, 1, 2, 3], [0, 1, 2, 3], [0, 1], [0, 1, 2]]
[{0: 0, 1: 1, 2: 2, 3: 3}, {0: 4, 1: 5, 2: 6, 3: 7}, {0: 8, 1: 9}, {0: 10, 1: 11, 2: 12}]
[{0: 0, 1: 1, 2: 2, 3: 3}, {4: 0, 5: 1, 6: 2, 7: 3}, {8: 0, 9: 1}, {10: 0, 11: 1, 12: 2}]


根据重新映射的点，重新映射边

In [56]:
rename_graphs_edges = []
for index, sub_graph_edges in enumerate(graphs_edges):
    rename_sub_graph_edges = []
    for edge in sub_graph_edges:
        rename_sub_graph_edges.append((rename_nodes_list_1[index][edge[0]], rename_nodes_list_1[index][edge[1]]))
#         print(f'当前为第{index}个子图', '原边为：', edge, '重命名边为：', (rename_nodes_list_1[index][edge[0]], rename_nodes_list_1[index][edge[1]]))
    rename_graphs_edges.append(rename_sub_graph_edges)

print(rename_graphs_edges)

[[(0, 1), (1, 2), (2, 3), (1, 3), (0, 3)], [(0, 1), (0, 2), (0, 3), (1, 3), (1, 2), (2, 3)], [(0, 1)], [(0, 2), (0, 1), (1, 2)]]


定义量子线路和哈密顿量

In [57]:
from mindquantum import *
import mindspore as ms
import mindspore.nn as nn
ms.set_context(mode=ms.PYNATIVE_MODE, device_target="CPU")

def build_hc(edges, para): 
    circ_ = Circuit()
    for i in edges:
        circ_ += ZZ(para).on(i[:2])
    circ_.barrier() 
    return circ_

def build_hb(nodes, para):
    circ_ = Circuit()
    for i in nodes:
        circ_ += RX(para).on(i)
    circ_.barrier() 
    return circ_

def build_ansatz(nodes, edges, p):    
    circ = Circuit()  
    for i in range(p):
        circ += build_hc(edges, f'g{i}')     
        circ += build_hb(nodes, f'b{i}')
    return circ

def build_ham(edges):
    ham = QubitOperator()
    for i in edges:
        ham += QubitOperator(f'Z{i[0]} Z{i[1]}') 
    return ham

# 当计算总图最大割时，需要使用带有权重的边
def build_ham_for_weighted_edges(weighted_edges):
    ham = QubitOperator()
    for i in weighted_edges:
        ham += QubitOperator(f'Z{i[0]} Z{i[1]}', i[2]) 
    return ham

采用 QAOA 计算各子图的最大割：

In [72]:
import numpy as np
import networkx as nx
import math

p = 4 # 量子线路层数
graphs_num = len(graphs_nodes)
left_nodes = [] # 记录分割方案，也就是每个子图中，哪些顶点被分类到左边，剩下的自然就在右边。

graphs_max_cut = 0 # 记录该方案下，整图的最大割数
for graph in range(graphs_num):
    print('当前子图边为：', rename_graphs_edges[graph])
    ham = Hamiltonian(build_ham(rename_graphs_edges[graph]))         
    init_state_circ = UN(H, rename_graphs_nodes[graph])  
    ansatz = build_ansatz(rename_graphs_nodes[graph], rename_graphs_edges[graph], p)  
    circ = init_state_circ + ansatz   

    sim = Simulator('mqvector', circ.n_qubits)
    grad_ops = sim.get_expectation_with_grad(ham, circ)
    net = MQAnsatzOnlyLayer(grad_ops)
    opti = nn.Adam(net.trainable_params(), learning_rate=0.05)
    train_net = nn.TrainOneStepCell(net, opti)
    for i in range(200):
        train_net()
        
    sim.reset()
    sim.apply_circuit(circ, pr=dict(zip(ansatz.params_name, net.weight.asnumpy())))
    state = sim.get_qs()
    distribution = [] # 坍缩到各基矢的概率分布
    for i in range(len(state)):
        distribution.append((state[i].conj()*state[i]).real)
        
    res =  "{0:b}".format(np.argmax(distribution)) # 找出概率最大的那个基矢序号，并将其转化为转化为二进制
    res = res.zfill(len(rename_graphs_nodes[graph])) # 前面补位，这样才能保证比特数正常
    res = res[::-1] # 翻转，然后就得到了类似 0101 这样标志着测量结果的二进制表示
    sub_graph_left_nodes = []
    for index, i in enumerate(res):
        if i == '0': # 如果是 0 则归类到左边，若为 1 则归类到右边
            sub_graph_left_nodes.append(index)
    print('当前子图归类到左边的顶点为：', sub_graph_left_nodes)
    
    g = nx.Graph()
    for edge in rename_graphs_edges[graph]:
        nx.add_path(g, [edge[0], edge[1]], weight=1)
    cut = nx.cut_size(g, sub_graph_left_nodes, weight='weight')
    graphs_max_cut += cut
    print('该子图的最大割为：', cut)
    print('')
    left_nodes.append(sub_graph_left_nodes)
print('各子图分类到左边的顶点为：', left_nodes)
print('各子图所累积的最大割数为：', graphs_max_cut)

当前子图边为： [(0, 1), (1, 2), (2, 3), (1, 3), (0, 3)]
当前子图归类到左边的顶点为： [0, 2]
该子图的最大割为： 4

当前子图边为： [(0, 1), (0, 2), (0, 3), (1, 3), (1, 2), (2, 3)]
当前子图归类到左边的顶点为： [1, 2]
该子图的最大割为： 4

当前子图边为： [(0, 1)]
当前子图归类到左边的顶点为： [1]
该子图的最大割为： 1

当前子图边为： [(0, 2), (0, 1), (1, 2)]
当前子图归类到左边的顶点为： [2]
该子图的最大割为： 2

各子图分类到左边的顶点为： [[0, 2], [1, 2], [1], [2]]
各子图所累积的最大割数为： 11


将各子图作为顶点，构建一个新的总图。
先找到子图的边，也就是子图之间的边，并进行重新映射。

In [67]:
inner_graph_edges = [i for ii in graphs_edges for i in ii]
inter_graph_edges = list(set(edges)-set(inner_graph_edges))
print('边总数为：', len(edges))
print('各子图内部边数为：', len(inner_graph_edges))
print('各子图间的边数为：', len(inter_graph_edges))

rename_inter_edges = []
for edge in inter_graph_edges:
    for index, nodes in enumerate(graphs_nodes):
        if edge[0] in nodes:
            node0 = index
    for index, nodes in enumerate(graphs_nodes):
        if edge[1] in nodes:
            node1 = index
    rename_inter_edges.append((node0, node1))
    
print('总图的边为：', rename_inter_edges)

边总数为： 21
各子图内部边数为： 15
各子图间的边数为： 6
总图的边为： [(1, 3), (0, 3), (2, 3), (1, 2), (0, 1), (0, 1)]


给新图边加上权重：当一个边的两个顶点在各自子图中被分类到同一边时，权重为 +1，否则为 -1；两个子图间都多个边时，合并为一个边，而权重累加。

In [100]:
import copy

sub_left_nodes = [] # 所有子图中，被分类到左边中的点
for index, nodes in enumerate(left_nodes):
    for j in nodes:
        pre_node = rename_nodes_list_0[index][j]
        sub_left_nodes.append(pre_node) 
        
# print(sub_left_nodes) 
# [0, 2, 5, 6, 9, 10, 12]

# print(inter_graph_edges)
# [(5, 11), (0, 12), (9, 10), (7, 8), (2, 4), (3, 5)]

weighted_inter_graph_edges = [] 
for edge in inter_graph_edges:
    value_0 = 1 if edge[0] in sub_left_nodes else -1
    value_1 = 1 if edge[1] in sub_left_nodes else -1
    value = value_0 * value_1
    weighted_inter_graph_edges.append([edge[0], edge[1], value])
    
# print(weighted_inter_graph_edges)
# [[5, 11, -1], [0, 12, 1], [9, 10, 1], [7, 8, 1], [2, 4, -1], [3, 5, -1]]

rename_inter_nodes = list(range(len(graphs_nodes)))
rename_inter_edges = []
for edge in weighted_inter_graph_edges:
    for index, nodes in enumerate(graphs_nodes):
        if edge[0] in nodes:
            node0 = index
    for index, nodes in enumerate(graphs_nodes):
        if edge[1] in nodes:
            node1 = index
    rename_inter_edges.append([node0, node1, edge[2]])
    
print(rename_inter_edges)

[[1, 3, -1], [0, 3, 1], [2, 3, -1], [1, 2, 1], [0, 1, -1], [0, 1, -1]]


In [101]:
import numpy as np
import networkx as nx
import math
p = 4

rename_inter_edges = [[1, 3, -1], [0, 3, 1], [2, 3, -1], [1, 2, 1], [0, 1, -1], [0, 1, -1]]

qubits_num = len(graphs_nodes)
ham = Hamiltonian(build_ham_for_weighted_edges(rename_inter_edges))         
init_state_circ = UN(H, list(range(qubits_num)))  
ansatz = build_ansatz(rename_inter_nodes, rename_inter_edges, p)  
circ = init_state_circ + ansatz   

sim = Simulator('mqvector', circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham, circ)
net = MQAnsatzOnlyLayer(grad_ops)
opti = nn.Adam(net.trainable_params(), learning_rate=0.05)
train_net = nn.TrainOneStepCell(net, opti)
for i in range(500):
    train_net()
print(rename_inter_edges)
sim.reset()
sim.apply_circuit(circ, pr=dict(zip(ansatz.params_name, net.weight.asnumpy())))
state = sim.get_qs()
distribution = []
for i in range(len(state)):
    distribution.append((state[i].conj()*state[i]).real)
print(np.argmax(distribution))
res =  "{0:b}".format(np.argmax(distribution))
res = res.zfill(4)
res = res[::-1]
overall_graph_left_nodes = []
for index, i in enumerate(res):
    if i == '0':
        overall_graph_left_nodes.append(index)
print('总图分类到左边的顶点为：', overall_graph_left_nodes)
g = nx.Graph()
for edge in rename_inter_edges:
    nx.add_path(g, [edge[0], edge[1]], weight=edge[2])
cut = nx.cut_size(g, overall_graph_left_nodes, weight='weight')
max_cut = cut + graphs_max_cut
print('最终的最大割数为：', max_cut)

[[1, 3, -1], [0, 3, 1], [2, 3, -1], [1, 2, 1], [0, 1, -1], [0, 1, -1]]
3
总图分类到左边的顶点为： [2, 3]
最终的最大割数为： 12


## 自验结果

下图展示了采用 QAOA in QAOA 算法，使用 4, 7, 11, 13 个量子比特的系统，在量子线路层数分别为 1，2，3，4，5 的情况下，求解顶点数为 15，每个节点有 4 个邻居随机正则图的结果。 **注：** 图中每一项数据都是对 10 个随机图分别求解并取平均的结果。

![result](./src/result.png)

该图通过运行 `main.ipynb` 中的所有语句得到，结论和论文图 FIG.5 (b) 一致。

**结论：** 通过以上复现结果，可见采用 QAOA in QAOA 算法，确实可以用小型量子计算机求解大型图的最大割问题。且量子计算机的规模越大，求解结果越接近真实值。复现结果与原文结论相一致。


## 自验环境

硬件：Windows 10 系统、 4U8G 256G 存储 

软件：Mindspore 1.6.0、Mindquantum 0.7.0

训练超参数：learning rate = 0.05、optimizer = Adam

